# Training and testing model for emotion detection

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
IMG_HEIGHT=48 
IMG_WIDTH = 48
batch_size=32

train_data_dir='data_selected/train/'
validation_data_dir='data_selected/test/'

train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(IMG_HEIGHT, IMG_WIDTH),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(IMG_HEIGHT, IMG_WIDTH),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

#Verify our generator by plotting a few faces and printing corresponding labels
class_labels=['Angry','Happy','Neutral','Sad','Surprise']

## Model definition

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


train_path = "data_selected/train/"
test_path = "data_selcted/test"

num_train_imgs = 0
for root, dirs, files in os.walk(train_path):
    num_train_imgs += len(files)
    
num_test_imgs = 0
for root, dirs, files in os.walk(test_path):
    num_test_imgs += len(files)

## Training the model

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

class ConfusionMatrixCallback(Callback):
    def __init__(self, validation_generator, freq=5):
        super().__init__()
        self.validation_generator = validation_generator
        self.freq = freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.freq == 0:
            predictions = self.model.predict(self.validation_generator)
            predicted_classes = np.argmax(predictions, axis=1)
            true_classes = self.validation_generator.classes
            cm = confusion_matrix(true_classes, predicted_classes)
            plt.figure(figsize=(10, 7))
            sns.heatmap(cm, annot=True, fmt="d")
            plt.title('Confusion Matrix')
            plt.ylabel('Actual labels')
            plt.xlabel('Predicted labels')
            plt.show()

In [ ]:
epochs=25

confusion_matrix_callback = ConfusionMatrixCallback(validation_generator, freq=5)

history=model.fit(train_generator,
                steps_per_epoch=num_train_imgs//batch_size,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=num_test_imgs//batch_size,
                    callbacks=[confusion_matrix_callback])

model.save('emotion_detection_model_selected_25epochs.h5')

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Testing the model

In [ ]:
from keras.models import load_model
from math import ceil
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

model = load_model('emotion_detection_model_selected.h5')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

predictions = model.predict(validation_generator, steps=ceil(validation_generator.samples / batch_size))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes

cm = confusion_matrix(true_classes, predicted_classes)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Accuracy: {accuracy}")

from sklearn.metrics import cohen_kappa_score

class_accuracy = 100 * cm.diagonal() / cm.sum(axis=1)
class_labels = list(validation_generator.class_indices.keys())

for i, accuracy in enumerate(class_accuracy):
    print(f"Dokładność klasy {class_labels[i]}: {accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels, zero_division=0))

print("Cohen's Kappa:", cohen_kappa_score(true_classes, predicted_classes))